<br>

<div align=center><font color=maroon size=6><b>TensorFlow 2 quickstart for experts</b></font></div>

<br>

<font size=4><b>References:</b></font>
1. TF2 official tutorials: <a href="https://www.tensorflow.org/tutorials" style="text-decoration:none;">TensorFlow Tutorials</a> 
    * `TensorFlow > Learn > TensorFlow Core > `Tutorials > <a href="https://www.tensorflow.org/tutorials/quickstart/advanced" style="text-decoration:none;">TensorFlow 2 quickstart for experts</a>
        * <a href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/advanced.ipynb#scrollTo=3wF5wszaj97Y" style="text-decoration:none;">Google Colab</a>

<br>
<br>
<br>

## Import TensorFlow into your program:

In [1]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [2]:
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.5.0


<br>
<br>

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0

In [4]:
# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

<br>
<br>

## Use `tf.data` to batch and shuffle the dataset:

In [5]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

<br>
<br>

## Build the `tf.keras` model using the Keras [model subclassing API](https://www.tensorflow.org/guide/keras#model_subclassing):

In [6]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)
        
        
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        # x = self.d2(x)
        
        return self.d2(x)

In [7]:
# Create an instance of the model
model = MyModel()

<br>
<br>

## Choose an optimizer and loss function for training:

In [8]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

<br>
<br>

## Select metrics

Select metrics to measure the loss and the accuracy of the model. These metrics accumulate the values over epochs and then print the overall result.

In [9]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

<br>
<br>

## Use `tf.GradientTape` to train the model:

In [10]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
        
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

<br>
<br>

## Test the model:

In [11]:
@tf.function
def test_step(images, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)
    
    test_loss(t_loss)
    test_accuracy(labels, predictions)

<br>
<br>

## Start training

In [12]:
EPOCHS = 5

for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_state()
    train_accuracy.reset_state()
    test_loss.reset_states()
    test_accuracy.reset_state()
    
    for images, labels in train_ds:
        train_step(images, labels)
        
    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)
    
    print(f'Epoch {epoch + 1}, '
          f'Loss: {train_loss.result()}, '
          f'Accuracy: {train_accuracy.result() * 100}, '
          f'Test Loss: {test_loss.result()}, '
          f'Test Accuracy: {test_accuracy.result() * 100}'
         )

Epoch 1, Loss: 0.13323673605918884, Accuracy: 96.0, Test Loss: 0.05857252702116966, Test Accuracy: 97.93000030517578
Epoch 2, Loss: 0.04166758060455322, Accuracy: 98.74500274658203, Test Loss: 0.05718867853283882, Test Accuracy: 98.07999420166016
Epoch 3, Loss: 0.02198532037436962, Accuracy: 99.29000091552734, Test Loss: 0.05285380035638809, Test Accuracy: 98.33999633789062
Epoch 4, Loss: 0.012502163648605347, Accuracy: 99.6050033569336, Test Loss: 0.058407824486494064, Test Accuracy: 98.44999694824219
Epoch 5, Loss: 0.007338435854762793, Accuracy: 99.76499938964844, Test Loss: 0.06373685598373413, Test Accuracy: 98.27999877929688


<br>
<br>

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials).

<br>
<br>
<br>